In [1]:
import pandas as pd

In [2]:
#load dataset
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
#here userr ID is of no use, so remove that
dataset=dataset.drop("User ID",axis=1)

In [7]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset[["Purchased"]]

In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid= { 'penalty': ['l1', 'l2', 'elasticnet', None] , 'class_weight': ['balanced', 'dict', None], 'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'], 'C':[1,10,20,100]}
grid= GridSearchCV(LogisticRegression(), param_grid, refit= True, verbose=3, n_jobs=-1, scoring= 'f1_weighted' )
grid.fit(X_train,y_train)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


C:\Anaconda3run\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
920 fits failed out of a total of 1440.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3run\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3run\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3run\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1168, in fit
    solver = _check_solver(self.solver, self.pe

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [1, 10, 20, 100],
                         'class_weight': ['balanced', 'dict', None],
                         'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='f1_weighted', verbose=3)

In [12]:
y_pred=grid.predict(X_test) 
y_pred

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1], dtype=int64)

In [13]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[79  6]
 [10 39]]


In [14]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134



In [15]:
grid.best_params_

{'C': 1, 'class_weight': None, 'penalty': 'l1', 'solver': 'saga'}

In [16]:
print("The best parameters are:", grid.best_params_)

The best parameters are: {'C': 1, 'class_weight': None, 'penalty': 'l1', 'solver': 'saga'}


In [17]:
re=grid.cv_results_

In [18]:
grid.best_score_

0.8287659267359453

In [19]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001593,0.000486,0.000000,0.000000,1,balanced,l1,lbfgs,"{'C': 1, 'class_weight': 'balanced', 'penalty'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,105
1,0.008182,0.001593,0.014159,0.001328,1,balanced,l1,liblinear,"{'C': 1, 'class_weight': 'balanced', 'penalty'...",0.777778,0.815037,0.756031,0.815056,0.906166,0.814014,0.051321,53
2,0.002195,0.001934,0.000000,0.000000,1,balanced,l1,newton-cg,"{'C': 1, 'class_weight': 'balanced', 'penalty'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,105
3,0.001598,0.000799,0.000000,0.000000,1,balanced,l1,newton-cholesky,"{'C': 1, 'class_weight': 'balanced', 'penalty'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,105
4,0.002802,0.001931,0.000000,0.000000,1,balanced,l1,sag,"{'C': 1, 'class_weight': 'balanced', 'penalty'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,0.010572,0.004486,0.000000,0.000000,100,None,None,liblinear,"{'C': 100, 'class_weight': None, 'penalty': No...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,105
284,0.016558,0.003921,0.016553,0.002408,100,None,None,newton-cg,"{'C': 100, 'class_weight': None, 'penalty': No...",0.804764,0.811321,0.682050,0.923510,0.922185,0.828766,0.089565,1
285,0.012169,0.004153,0.015758,0.000746,100,None,None,newton-cholesky,"{'C': 100, 'class_weight': None, 'penalty': No...",0.804764,0.811321,0.682050,0.923510,0.922185,0.828766,0.089565,1
286,0.009578,0.003062,0.012164,0.001592,100,None,None,sag,"{'C': 100, 'class_weight': None, 'penalty': No...",0.804764,0.811321,0.682050,0.923510,0.922185,0.828766,0.089565,1


In [20]:
from sklearn.metrics import roc_auc_score
roc_score= roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])
roc_score

0.9493397358943578

In [22]:
pre_input= sc.transform([[35,60000,0]])
pre_input

C:\Anaconda3run\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-0.27290378, -0.27655878, -0.99250926]])

In [23]:
result= grid.predict(pre_input)
result

array([0], dtype=int64)